In [ ]:
import pandas as pd
import numpy as np

# Create test data 2024-01-01 to 2024-12-31
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/keepa_data/final_df.csv')
df[['query','category','product group','brand']].dropna()

def get_price_metrics(price_data):
    price_data = price_data.dropna()
    price_data = price_data.sort_values(['query', 'date'])

    results = []
    for query in price_data['query'].unique():
        query_data = price_data[price_data['query'] == query].copy()
        price_col = [col for col in query_data.columns if col not in ['date', 'query']][0]

        # Fill missing prices
        query_data[price_col] = query_data[price_col].ffill().bfill()

        # Calculate price changes across ALL observations (including cross-day)
        query_data['price_changed'] = (query_data[price_col] != query_data[price_col].shift(1)).astype(int)
        query_data['price_diff'] = query_data[price_col].diff()
        query_data['price_upwards'] = (query_data['price_diff'] > 0).astype(int)
        query_data['price_downwards'] = (query_data['price_diff'] < 0).astype(int)

        # Set first row to 0 for all change metrics
        query_data.iloc[0, query_data.columns.get_loc('price_changed')] = 0
        query_data.iloc[0, query_data.columns.get_loc('price_diff')] = 0
        query_data.iloc[0, query_data.columns.get_loc('price_upwards')] = 0
        query_data.iloc[0, query_data.columns.get_loc('price_downwards')] = 0

        # Convert to date for daily aggregation
        query_data['date'] = pd.to_datetime(query_data['date']).dt.date

        # Group by date and aggregate
        daily_agg = query_data.groupby('date').agg({
            price_col: 'mean',
            'price_changed': 'sum',
            'price_upwards': 'sum',
            'price_downwards': 'sum',
            'price_diff': 'sum'  # This becomes momentum
        })

        # Calculate variance and range for each day
        for date in daily_agg.index:
            day_data = query_data[query_data['date'] == date]

            # Realized variance from price changes
            if len(day_data) > 1:
                price_changes = day_data[price_col].diff().dropna()
                if len(price_changes) > 0:
                    demeaned_changes = price_changes - price_changes.mean()
                    realized_variance = np.sum(demeaned_changes ** 2)
                else:
                    realized_variance = 0
            else:
                realized_variance = 0

            # Intraday range
            intraday_range = (day_data[price_col].max() - day_data[price_col].min()) ** 2

            results.append({
                'date': date,
                'query': query,
                f'{price_col}': daily_agg.loc[date, price_col],
                f'{price_col}_frequency': daily_agg.loc[date, 'price_changed'],
                f'{price_col}_upwards': daily_agg.loc[date, 'price_upwards'],
                f'{price_col}_downwards': daily_agg.loc[date, 'price_downwards'],
                f'{price_col}_momentum': daily_agg.loc[date, 'price_diff'],
                f'{price_col}_realized_variance': realized_variance,
                f'{price_col}_intraday_range': intraday_range
            })

    return pd.DataFrame(results)

# Reviews function 
def get_daily_reviews(review_data):
    review_data = review_data.sort_values(['query', 'date'])

    results = []
    for query in review_data['query'].unique():
        query_data = review_data[review_data['query'] == query].copy()

        # Replace -1 with NaN and fill
        query_data['total_count_reviews'] = query_data['total_count_reviews_raw'].replace(-1, np.nan)
        query_data['total_count_reviews'] = query_data['total_count_reviews'].ffill().bfill()

        # Calculate review differences (including cross-day)
        query_data['reviews'] = query_data['total_count_reviews'].diff()

        # Handle first row and negative values
        query_data.loc[query_data.index[0], 'reviews'] = 0
        query_data['reviews'] = query_data['reviews'].apply(lambda x: max(0, x) if pd.notnull(x) else 0)

        # IQR outlier detection before aggregation
        if len(query_data) > 3:
            filtered_reviews = query_data[query_data['reviews'] > 0]
            if len(filtered_reviews) > 0:
                q1 = filtered_reviews['reviews'].quantile(0.25)
                q3 = filtered_reviews['reviews'].quantile(0.75)
                iqr = q3 - q1

                upper_bound = q3 + 1.5 * iqr

                outlier_mask = (query_data['reviews'] > upper_bound)

                if outlier_mask.any():
                    median_reviews = query_data['reviews'].median()
                    query_data.loc[outlier_mask, 'reviews'] = median_reviews

        # Convert to date and aggregate to daily sum
        query_data['date'] = pd.to_datetime(query_data['date']).dt.date

        daily_reviews = query_data.groupby('date')['reviews'].sum().reset_index()
        daily_reviews['query'] = query

        results.append(daily_reviews)

    return pd.concat(results, ignore_index=True)

# Rank function 
def get_daily_ranks(rank_data):
    rank_data = rank_data.copy()

    # Convert to date for daily aggregation
    rank_data['date'] = pd.to_datetime(rank_data['date']).dt.date

    # Group by query and date, calculate mean rank
    daily_ranks = rank_data.groupby(['query', 'date'])['store_rank'].mean().reset_index()

    # Round to nearest integer
    daily_ranks['store_rank'] = daily_ranks['store_rank'].round(0)

    return daily_ranks

# Create date series for 2024
an_df = an_df.sort_values(by=['query', 'date'])

dates_2024 = pd.date_range('2024-01-01', '2024-12-31', freq='D')
date_df = pd.DataFrame({'date': dates_2024})
date_df['date'] = pd.to_datetime(date_df['date']).dt.date
# Process amazon_price

amazon_price_list = []

for query in an_df['query'].unique():
    query_data = an_df[an_df['query'] == query].copy()
    price_metrics = get_price_metrics(query_data[['date', 'query', 'amazon_price']])

    query_price_df = date_df.merge(price_metrics, on='date', how='left')
    query_price_df['query'] = query

    # Fill missing values after merge
    query_price_df['amazon_price'] = query_price_df['amazon_price'].ffill().bfill()
    query_price_df['amazon_price_frequency'] = query_price_df['amazon_price_frequency'].fillna(0)
    query_price_df['amazon_price_upwards'] = query_price_df['amazon_price_upwards'].fillna(0)
    query_price_df['amazon_price_downwards'] = query_price_df['amazon_price_downwards'].fillna(0)
    query_price_df['amazon_price_momentum'] = query_price_df['amazon_price_momentum'].fillna(0)
    query_price_df['amazon_price_realized_variance'] = query_price_df['amazon_price_realized_variance'].fillna(0)
    query_price_df['amazon_price_intraday_range'] = query_price_df['amazon_price_intraday_range'].fillna(0)

    amazon_price_list.append(query_price_df)


amazon_price_final = pd.concat(amazon_price_list, ignore_index=True)

review_list = []

for query in an_df['query'].unique():
    query_data = an_df[an_df['query'] == query].copy()

    # Get review metrics for this query
    review_metrics = get_daily_reviews(query_data[['date', 'query', 'total_count_reviews_raw']])

    # Merge with full date range to ensure all dates present
    query_review_df = date_df.merge(review_metrics, on='date', how='left')
    query_review_df['query'] = query
    query_review_df['reviews'] = query_review_df['reviews'].fillna(0)
    review_list.append(query_review_df)

review_final = pd.concat(review_list, ignore_index=True)

rank_list = []

for query in an_df['query'].unique():
    query_data = an_df[an_df['query'] == query].copy()

    rank_metrics = get_daily_ranks(query_data[['date', 'query', 'store_rank']])

    # Merge with full date range to ensure all dates present
    query_rank_df = date_df.merge(rank_metrics, on='date', how='left')
    query_rank_df['query'] = query
    query_rank_df['store_rank'] = query_rank_df.groupby('query')['store_rank'].ffill().bfill()
    rank_list.append(query_rank_df)

rank_final = pd.concat(rank_list, ignore_index=True)
print(f"Rank final shape: {rank_final.shape}")

# Merge all amazon_price data
print("\n=== Merging amazon_price final dataframe ===")
amazon_final_df = amazon_price_final.merge(review_final[['date', 'query', 'reviews']], on=['date', 'query'], how='left')
amazon_final_df = amazon_final_df.merge(rank_final[['date', 'query', 'store_rank']], on=['date', 'query'], how='left')
amazon_final_df.to_csv('/content/drive/MyDrive/transformed_data/amazon_pricing.csv',index=False)
print(f"Amazon final df shape: {amazon_final_df.shape}")
